### Домашняя работа к уроку № 3:
### Системы управления базами данных MongoDB и Кликхаус в Python

1. Установите MongoDB на локальной машине, а также зарегистрируйтесь в онлайн-сервисе. https://www.mongodb.com/ https://www.mongodb.com/products/compass
2. Загрузите данные который вы получили на предыдущем уроке путем скрейпинга сайта с помощью Buautiful Soup в MongoDB и создайте базу данных и коллекции для их хранения.
3. Поэкспериментируйте с различными методами запросов.
4. Зарегистрируйтесь в ClickHouse.
5. Загрузите данные в ClickHouse и создайте таблицу для их хранения.

#### Выполнение задачи №2: Загрузите данные который вы получили на предыдущем уроке путем скрейпинга сайта с помощью Buautiful Soup в MongoDB и создайте базу данных и коллекции для их хранения.

In [1]:
import json
from pymongo import MongoClient

In [5]:
# Чтение данных из JSON файла
with open("books_from_books.toscrape.com.json", "r", encoding='utf-8') as file:
    data = json.load(file)

In [7]:
### Проверка загрузки на примере первой записи
data[0]

{'Название': 'A Light in the Attic',
 'Цена в фунтах стерлингов': 51.77,
 'Количество в наличии': 22,
 'Описание': "\n    It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock

In [8]:
# Соединение с MongoDB
client = MongoClient('localhost', 27017)
db = client['my_books_database']

In [22]:
# Создание коллекции на основании данных из JSON файла
collection_name = "Список книг от 12-03-2024"
for item in data:
    collection = db[collection_name]
    collection.insert_one(item)

client.close()

#### Выполнение задачи №3: Поэкспериментируйте с различными методами запросов.

In [29]:
# Установление связи с MongoDB
client = MongoClient('localhost', 27017)
db = client['my_books_database']
collection = db['Список книг от 12-03-2024']

In [24]:
#### Просмотр количества документов в коллекции
document_count = collection.count_documents({})
print("Количества документов в коллекции:", document_count)

Количества документов в коллекции: 1000


In [40]:
# Запрос количества и названий книг, дороже 50 фунтов
query = {"Цена в фунтах стерлингов": {"$gt": 59.0}}

# Получения документов, соответствующих запросу
documents = collection.find(query)

print("Количество книг дороже 59 фунтов - ", collection.count_documents(query), ":")

# Отображение названий книг
for document in documents:
    print("Название книги:", document['Название'])

Количество книг дороже 59 фунтов -  12 :
Название книги: Thomas Jefferson and the Tripoli Pirates: The Forgotten War That Changed American History
Название книги: The Gray Rhino: How to Recognize and Act on the Obvious Dangers We Ignore
Название книги: The Diary of a Young Girl
Название книги: Boar Island (Anna Pigeon #19)
Название книги: The Improbability of Love
Название книги: The Man Who Mistook His Wife for a Hat and Other Clinical Tales
Название книги: The Barefoot Contessa Cookbook
Название книги: Last One Home (New Beginnings #1)
Название книги: The Perfect Play (Play by Play #1)
Название книги: The Bone Hunters (Lexy Vaughan & Steven Macaulay #2)
Название книги: Life Without a Recipe
Название книги: Civilization and Its Discontents


In [42]:
### Запрос для определения максимальной и минимальной цены книги в коллекции

# Определение конвеера для агрегации данных
pipeline = [
    {"$group": {"_id": None, "max_price": {"$max": "$Цена в фунтах стерлингов"}, "min_price": {"$min": "$Цена в фунтах стерлингов"}}}
]

# Применение конвеера
result = list(collection.aggregate(pipeline))

# Получение минимального и максимального значений
max_price = result[0]["max_price"]
min_price = result[0]["min_price"]

# Печать результатов
print("Минимальная цена:", min_price)
print("Максимальная цена:", max_price)


Минимальная цена: 10.0
Максимальная цена: 59.99


In [56]:
# Создание конвеера для разбиения всех книг по стоимости на три категории: низкая стоимость (дешевле 15 фунтов), средня стоимость (от 15 до 25 фунтов) и высокая стоимость (выше 25 фунтов). Затем подсчет количества книг в каждой категории агрегацией по полю "Количество в наличии"
pipeline_by_group = [
    {
        "$group": {
            "_id": {
                "$cond": {
                    "if": {"$lte": ["$Цена в фунтах стерлингов", 15]},
                    "then": "Низкая стоимость",
                    "else": {
                        "$cond": {
                            "if": {"$lte": ["$Цена в фунтах стерлингов", 25]},
                            "then": "Средная стоимость",
                            "else": "Высокая стоимость"
                        }
                    }
                }
            },
            "Количество в наличии": {"$sum": "$Количество в наличии"}
        }
    }
]

# Выполнение конвеера
result_by_group = list(collection.aggregate(pipeline_by_group))

# Печать результатов
for category in result_by_group:
    print("Категория по стоимости:", category["_id"])
    print("Количество в наличии:", category["Количество в наличии"])
    print()

# Создание конвеера для определения суммарного количества всех книг по полю "Количество в наличии"
pipeline_total_count = [
    {
        "$group": {
            "_id": None,
            "total_count": {"$sum": "$Количество в наличии"}
        }
    }
]

# Выполнение конвеера
result_total_count = list(collection.aggregate(pipeline_total_count))

# Отображение результата
total_count = result_total_count[0]['total_count'] if result_total_count else 0
print("Общее количество книг в наличии:", total_count) #### совпадает с суммой книг в наличии по категориям стоимости

Категория по стоимости: Высокая стоимость
Количество в наличии: 5951

Категория по стоимости: Средная стоимость
Количество в наличии: 1722

Категория по стоимости: Низкая стоимость
Количество в наличии: 912

Общее количество книг в наличии: 8585
